In [1]:
%matplotlib inline

import numpy as np 
import matplotlib.pyplot as plt

import theano
import theano.tensor as tt
from theano.tensor import fft
import theano.sparse

import pymc3 as pm

# add the gridding path 
import sys
sys.path.append("/home/ian/Research/Disks/MillionPoints/million-points-of-light")

import gridding

# convert from arcseconds to radians
arcsec = np.pi / (180.0 * 3600) # [radians]  = 1/206265 radian/arcsec

In [2]:
def sky_plane(alpha, dec, a=1, delta_alpha=1.0*arcsec, delta_delta=1.0*arcsec, sigma_alpha=1.0*arcsec,
              sigma_delta=1.0*arcsec, Omega=0.0):
    '''
    alpha: ra (in radians)
    delta: dec (in radians)
    a : amplitude
    delta_alpha : offset (in radians)
    delta_dec : offset (in radians)
    sigma_alpha : width (in radians)
    sigma_dec : width (in radians)
    Omega : position angle of ascending node (in degrees east of north)
    '''

    return a * np.exp(-( (alpha - delta_alpha)**2/(2 * sigma_alpha**2) + \
                        (dec - delta_delta)**2/(2 * sigma_delta**2)))


def fourier_plane(u, v, a=1, delta_alpha=1.0*arcsec, delta_delta=1.0*arcsec, sigma_alpha=1.0*arcsec,
              sigma_delta=1.0*arcsec, Omega=0.0):
    '''
    Calculate the Fourier transform of the Gaussian. Assumes u, v in kλ.
    '''

    # convert back to lambda
    u = u * 1e3
    v = v * 1e3

    return 2 * np.pi * a * sigma_alpha * sigma_delta * np.exp(- 2 * np.pi**2 * \
                (sigma_alpha**2 * u**2 + sigma_delta**2 * v**2) - 2 * np.pi * 1.0j * \
                                                    (delta_alpha * u + delta_delta * v))


# the gradients
def dV_ddelta_alpha(u, v, a=1, delta_alpha=1.0*arcsec, delta_delta=1.0*arcsec, sigma_alpha=1.0*arcsec,
              sigma_delta=1.0*arcsec, Omega=0.0):
    
    
    return -2 * np.pi * 1j * u * fourier_plane(u*1e-3, v*1e-3, a, delta_alpha, delta_delta, sigma_alpha,
              sigma_delta, Omega)


def dV_ddelta_delta(u, v, a=1, delta_alpha=1.0*arcsec, delta_delta=1.0*arcsec, sigma_alpha=1.0*arcsec,
              sigma_delta=1.0*arcsec, Omega=0.0):
    
    
    return -2 * np.pi * 1j * v * fourier_plane(u*1e-3, v*1e-3, a, delta_alpha, delta_delta, sigma_alpha,
              sigma_delta, Omega)


In [3]:
def fftspace(width, N):
    '''Oftentimes it is necessary to get a symmetric coordinate array that spans ``N``
     elements from `-width` to `+width`, but makes sure that the middle point lands
     on ``0``. The indices go from ``0`` to ``N -1.``
     `linspace` returns  the end points inclusive, wheras we want to leave out the
     right endpoint, because we are sampling the function in a cyclic manner.'''

    assert N % 2 == 0, "N must be even."

    dx = width * 2.0 / N
    xx = np.empty(N, np.float)
    for i in range(N):
        xx[i] = -width + i * dx
    
    return xx

In [4]:
# Let's plot this up and see what it looks like 

N_alpha = 128
N_dec = 128
img_radius = 15.0 * arcsec


# full span of the image
ra = fftspace(img_radius, N_alpha) # [arcsec]
dec = fftspace(img_radius, N_dec) # [arcsec]

In [5]:
# calculate the maximum u and v points that our image grid can sample 
dRA = (2 * img_radius) / N_alpha # radians
max_baseline = 1 / (2 * dRA) * 1e-3 # kilolambda, nyquist rate
print(max_baseline) # kilolambda

440.03158666047227


In [6]:
# create some fake data

N_vis = 100 # number of data points 

# the fake baselines where the Visibility function is sampled 
u_data = np.random.normal(loc=0, scale=0.1 * max_baseline, size=N_vis)
v_data = np.random.normal(loc=0, scale=0.1 * max_baseline, size=N_vis)

data_points = np.array([u_data, v_data]).T

# create a dataset of a Gaussian
data_only = fourier_plane(u_data, v_data)

# add some noise
noise = 1e-12 * np.ones(N_vis) # Jy
noise_draw = np.random.normal(loc=0, scale=noise, size=(N_vis)) + \
    np.random.normal(loc=0, scale=noise, size=(N_vis)) * 1.0j 
data_values = data_only + noise_draw

In [7]:
# create fixed quantities that we can pre-calculate in numpy before stuffing into the Theano part 

# the image plane grid (fixed throughout the problem)
XX, YY = np.meshgrid(np.fft.fftshift(ra), np.fft.fftshift(dec))

# the image-plane taper (fixed throughout problem)
corrfun_mat = gridding.corrfun_mat(np.fft.fftshift(ra), np.fft.fftshift(dec))

# the u and v coordinates of the RFFT output (also fixed throughout problem)
u_coords = np.fft.rfftfreq(N_alpha, d=(2 * img_radius)/N_alpha) * 1e-3  # convert to [kλ]
v_coords = np.fft.fftfreq(N_dec, d=(2 * img_radius)/N_dec) * 1e-3  # convert to [kλ]

# calculate the C_real and C_imag matrices
# these are scipy csc sparse matrices that will be stuffed into Theano objects
C_real, C_imag = gridding.calc_matrices(data_points, u_coords, v_coords)

# Sampling in PyMC3 

In [8]:
with pm.Model() as model:
    
    # create input grid as a shared variable
    # NOTE that these must be `fftshifted` already.
    # add an extra dimension for the later packing into the rfft
    alpha = theano.shared(XX[np.newaxis,:])
    dalpha = abs(alpha[0,0,1] - alpha[0,0,0])
    delta = theano.shared(YY[np.newaxis,:])
    ddelta = abs(delta[0,1,0] - delta[0,0,0])

    # 2) store the image premultiply matrix as a shared variable
    corrfun = theano.shared(corrfun_mat)

    # 3) store the CSC matrices that interpolate the RFFT grid
    C_real_sparse = theano.sparse.CSC(C_real.data, C_real.indices, C_real.indptr, C_real.shape)
    C_imag_sparse = theano.sparse.CSC(C_imag.data, C_imag.indices, C_imag.indptr, C_imag.shape)

    # Define the PyMC3 model parameters, which are just for the image plane model
    a = pm.Uniform("a", lower=0.0, upper=10.0)
    delta_alpha = pm.Uniform("delta_alpha", lower=-1*arcsec, upper=2*arcsec)
    delta_delta = pm.Uniform("delta_delta", lower=-1*arcsec, upper=2*arcsec)
    sigma_alpha = pm.Uniform("sigma_alpha", lower=0.5*arcsec, upper=1.5*arcsec)
    sigma_delta = pm.Uniform("sigma_delta", lower=0.5*arcsec, upper=1.5*arcsec)
    

    # Calculate the sky-plane model
    I = a * tt.exp(-(alpha - delta_alpha)**2/(2 * sigma_alpha**2) - (delta - delta_delta)**2/(2 * sigma_delta**2))
    # since the input coordinates were already shifted, then this is too
    # I shape should be (1, N_dec, N_alpha)

    # taper the image with the gridding correction function
    # this should broadcast OK, since the trailing image dimensions match
    I_tapered = I * corrfun

    # output from the RFFT is (1, N_delta, N_alpha//2 + 1, 2)
    rfft = dalpha * ddelta * fft.rfft(I_tapered, norm=None)  

    # flatten the RFFT output appropriately for the interpolation, taking the real and imag parts separately
    vis_real = rfft[0, :, :, 0].flatten() # real values 
    vis_imag = rfft[0, :, :, 1].flatten() # imaginary values

    # interpolate the RFFT to the baselines by a sparse matrix multiply
    interp_real = theano.sparse.dot(C_real_sparse, vis_real)
    interp_imag = theano.sparse.dot(C_imag_sparse, vis_imag)
    
    real_print = tt.printing.Print('interp_real')(interp_real)
    imag_print = tt.printing.Print('interp_imag')(interp_imag)
    
    # condition on the real and imaginary observations
    pm.Normal("obs_real", mu=interp_real, sd=noise, observed=np.real(data_values))
    pm.Normal("obs_imag", mu=interp_imag, sd=noise, observed=np.imag(data_values))

interp_real __str__ = [ 2.33566954e-10  3.05743427e-12  2.37373454e-11  7.09862675e-10
 -4.40691898e-12  5.06441324e-10  4.81357284e-10  6.85717304e-11
  1.31202017e-10 -3.21184266e-12  2.32175379e-11  7.27607910e-10
  4.71285043e-10  3.58106876e-10  1.24269360e-10 -4.87608370e-13
  3.04289652e-11  4.95877744e-12  5.67463235e-10  2.98161059e-10
  3.88761947e-10  5.50349155e-11  2.66576245e-10  5.24814199e-10
  1.50129320e-12 -4.73257001e-14  5.97913301e-10  6.02005685e-10
  4.54960499e-10  2.79730044e-10  2.96666892e-12  3.01180829e-11
  5.54526041e-11 -5.87759284e-12  6.81802888e-10  5.59049048e-10
  1.12407395e-10  1.37801339e-10  4.10264865e-11  3.39029415e-12
  2.65891590e-10  3.25083580e-10  1.52959964e-10  1.91163638e-11
  1.73315359e-10  6.58880591e-10  3.73895878e-10  1.65413956e-10
  2.05963652e-10  1.25900840e-10  1.29177715e-10  5.93840918e-12
 -9.40125290e-13  6.07234487e-10  4.37313993e-10  3.79585446e-12
  3.40276573e-11  6.20852147e-12  6.28300143e-10  1.26145293e-10
  2

In [10]:
model.basic_RVs

[a_interval__,
 delta_alpha_interval__,
 delta_delta_interval__,
 sigma_alpha_interval__,
 sigma_delta_interval__,
 obs_real,
 obs_imag]

In [11]:
model.unobserved_RVs

[a_interval__,
 delta_alpha_interval__,
 delta_delta_interval__,
 sigma_alpha_interval__,
 sigma_delta_interval__,
 a,
 delta_alpha,
 delta_delta,
 sigma_alpha,
 sigma_delta]

In [12]:
model.free_RVs

[a_interval__,
 delta_alpha_interval__,
 delta_delta_interval__,
 sigma_alpha_interval__,
 sigma_delta_interval__]

In [9]:
with model:
    trace = pm.sample(draws=1000, tune=1000, chains=1)

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...


ValueError: shapes (100,) and (8320,) not aligned: 100 (dim 0) != 8320 (dim 0)